In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#solution written by Blaise Moses for FreeCodeCamp Machine Learning with Python certification project

dataset.isna().sum() # test to make sure no null values in the dataset that need to be cleaned up
print(dataset.head())

CATEGORICAL=['sex', 'smoker', 'region']
NUMERIC = ['age', 'bmi', 'children']

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_dataset = pd.get_dummies(train_dataset, columns=CATEGORICAL)
test_dataset = pd.get_dummies(test_dataset,columns=CATEGORICAL)


train_dataset = train_dataset.astype(np.float32)
test_dataset = test_dataset.astype(np.float32)

#sns.pairplot(train_dataset, y_vars = 'expenses', x_vars=['sex_male', 'sex_female','smoker_yes', 'smoker_no','age','bmi','children', 'region_northeast', 'region_northwest','region_southeast','region_southwest'], diag_kind='kde')
# after plotting pairwise relationships, it seems like the age, bmi , and smoking status seem to have the most linear relationships with what a person's expense will be
# region also seems to play a role but the south and the north seem to show the same trend regardless of east or west, so I will include one of each to account for this

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

#train_dataset = train_dataset[['age','bmi','smoker_yes','smoker_no','region_southeast', 'region_northeast', 'region_southwest', 'region_northwest']]
#test_dataset = test_dataset[['age','bmi','smoker_yes','smoker_no','region_southeast', 'region_northeast', 'region_southwest', 'region_northwest']]
# experimented with what values to include but model performs better with full dataset so clearly at least some correlation in expense and all features, thus
# using full datasets for final product.

#normalizing the dataset to remove bias of larger valued labels in training, made a huge improvement in convergence speed
normalizer = layers.Normalization()
normalizer.adapt(test_dataset)

#final model , trains in 22ish seconds and has a mae just under 3000(2800ish)
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    #layers.Dropout(0.2),
    #layers.Dense(16, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)),
    #layers.Dropout(0.2),
    # layers.Dense(8, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)),
    # layers.Dropout(0.2),
    # layers.Dense(4, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)),
    # layers.Dropout(0.2), experimented with different layer quantities, too many hurts performance, so weened off.
    # normalized dataset requires far less layering than standard
    layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss='mae',
              metrics=['mae', 'mse'])

model.fit(train_dataset, train_labels, epochs=30, batch_size=5, validation_split=0.2, verbose=2)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
